In [1]:
import json # library to handle JSON files

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

import pandas as pd

import time

from bs4 import BeautifulSoup

## 获取北京市行政区划

In [2]:
# 获取北京市行政区划
#https://en.wikipedia.org/wiki/Beijing#Politics_and_government
#https://zh.wikipedia.org/wiki/%E5%8C%97%E4%BA%AC%E5%B8%82#%E7%B6%93%E6%BF%9F%E6%95%B8%E6%93%9A

In [37]:
df_divisions_bj = pd.read_csv("data/Administrative_divisions_BeiJing.csv")

In [38]:
df_divisions_bj.head()

,Division_code,Division,Area_in_km2,Total_population_2010,Urban_area_Population_2010,Seat,Postal_code,Subdistricts,Towns,Townships,Residential_communities,Villages,区划名称,政府驻地,划归北京时间,人口,面积,人口密度
0,110101,Dongcheng,41.82,919253,NaN,Jingshan Subdistrict,100000,17,,,216,,东城区,景山街道,共和国前,919000,42.0,21881
1,110102,Xicheng,50.33,1243315,NaN,Jinrong Street Subdistrict,100000,15,,,259,,西城区,金融街街道,共和国前,1243000,51.0,24372
2,110105,Chaoyang,454.78,3545137,3532257.0,Chaowai Subdistrict,100000,24,,19,358,5,朝阳区,朝外街道,共和国前,3545000,470.8,7530
3,110106,Fengtai,305.53,2112162,2098632.0,Fengtai Subdistrict,100000,16,2,3,254,73,丰台区,丰台街道,共和国前,2112000,304.2,6943
4,110107,Shijingshan,84.38,616083,NaN,Lugu Subdistrict,100000,9,,,130,,石景山区,鲁谷街道,共和国前,616000,89.8,6860


In [5]:
Divisions=df_divisions_bj['区划名称'].values

In [6]:
Divisions

array(['东城区', '西城区', '朝阳区', '丰台区', '石景山区', '海淀区', '门头沟区', '房山区', '通州区',
       '顺义区', '昌平区', '大兴区', '怀柔区', '平谷区', '密云区', '延庆区'], dtype=object)

## 获取每个区的农业银行坐标

In [34]:
keyword="工商银行"
tag="金融"
#region="海淀区"
ak="10ykqahdsmqaYOkrLgPzNNc6QAD2HOSZ"
page_size='20'
page_num=0

In [10]:
def call_api(url):
    while True:
        try:
            results = requests.get(url).json()
            if results['status']==0:
                break
        except:
            print("调用接口失败")
    return results

In [27]:
# 获取结果条数
def get_venues_num(keyword,tag,region,page_size,page_num,ak):
    url = 'http://api.map.baidu.com/place/v2/search?\
            query={}&tag={}&region={}&page_size={}&page_num={}&output=json&ak={}'.format(
               keyword,
               tag,
               region,
               page_size,
               page_num,
               ak)
    
    results=call_api(url)
    venues_num=results['total']
    
    return venues_num   

In [28]:
# 获取查询结果
def get_venues(keyword,tag,region,page_size,page_num,ak):
    
    #先计算结果总条数
    venues_num = get_venues_num(keyword,tag,region,page_size,page_num,ak)
    #用总条数计算结果页数
    page_num_all=venues_num//20 + 1
    print("{} 区域内,{}-{}个数：{},总页数{}".format(region,keyword,tag,venues_num,page_num_all))
    
    
    results = pd.DataFrame()
    for num in range(page_num_all):
        #print("完成第{}页".format(num))
        url = 'http://api.map.baidu.com/place/v2/search?query={}&tag={}&region={}&page_size={}&page_num={}&output=json&ak={}'.format(
           keyword,
           tag,
           region,
           page_size,
           num,
           ak)
        result = call_api(url)
        #time.sleep(1)
        venues=result['results']
        nearby_venues = json_normalize(venues) # flatten JSON
        #nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
        results=pd.concat([results,nearby_venues],ignore_index=True)
        #print(results.shape)
        
    return results

In [35]:
# 遍历北京市所有区
results = pd.DataFrame()
div_num=len(Divisions)

for i in range(div_num):
    region = Divisions[i]
    print("开始处理{}-{}/{}".format(i+1,region,div_num))
    
    
    # 获取单个区的银行坐标
    result = get_venues(keyword,tag,region,page_size,page_num,ak)
    #time.sleep(1)
    #venues=result['results']
    #nearby_venues = json_normalize(venues) # flatten JSON
    
    #处理列名
    result.columns = [col.split(".")[-1] for col in result.columns]
    #行并结果
    results=pd.concat([results,result],ignore_index=True)
    print("处理{}-{}/{}完成，{}-{} 数据集维度：{}".format(i+1,region,div_num,keyword,tag,results.shape))   
    
    #各区数据持久化
    df_name="data/{}_bj_{}_{}_divisions.csv".format(region,keyword,tag)
    result.to_csv(df_name,index=False)

#汇总数据持久化    
df_name="data/{}_bj_{}_{}_divisions.csv".format("all",keyword,tag)
results.to_csv(df_name,index=False)
print("{}-{}-{} 数据集维度：{}".format("北京市",keyword,tag,results.shape)) 

开始处理1-东城区/16
东城区 区域内,工商银行-金融个数：120,总页数7


/home/machinelearning/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:26: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



处理1-东城区/16完成，工商银行-金融 数据集维度：(120, 11)
开始处理2-西城区/16
西城区 区域内,工商银行-金融个数：199,总页数10
处理2-西城区/16完成，工商银行-金融 数据集维度：(319, 11)
开始处理3-朝阳区/16
朝阳区 区域内,工商银行-金融个数：248,总页数13
处理3-朝阳区/16完成，工商银行-金融 数据集维度：(567, 11)
开始处理4-丰台区/16
丰台区 区域内,工商银行-金融个数：124,总页数7
处理4-丰台区/16完成，工商银行-金融 数据集维度：(691, 11)
开始处理5-石景山区/16
石景山区 区域内,工商银行-金融个数：65,总页数4
处理5-石景山区/16完成，工商银行-金融 数据集维度：(756, 11)
开始处理6-海淀区/16
海淀区 区域内,工商银行-金融个数：229,总页数12
处理6-海淀区/16完成，工商银行-金融 数据集维度：(985, 11)
开始处理7-门头沟区/16
门头沟区 区域内,工商银行-金融个数：18,总页数1
处理7-门头沟区/16完成，工商银行-金融 数据集维度：(1003, 11)
开始处理8-房山区/16
房山区 区域内,工商银行-金融个数：55,总页数3
处理8-房山区/16完成，工商银行-金融 数据集维度：(1058, 11)
开始处理9-通州区/16
通州区 区域内,工商银行-金融个数：55,总页数3
处理9-通州区/16完成，工商银行-金融 数据集维度：(1113, 11)
开始处理10-顺义区/16
顺义区 区域内,工商银行-金融个数：43,总页数3
处理10-顺义区/16完成，工商银行-金融 数据集维度：(1156, 11)
开始处理11-昌平区/16
昌平区 区域内,工商银行-金融个数：46,总页数3
处理11-昌平区/16完成，工商银行-金融 数据集维度：(1202, 11)
开始处理12-大兴区/16
大兴区 区域内,工商银行-金融个数：57,总页数3
处理12-大兴区/16完成，工商银行-金融 数据集维度：(1259, 11)
开始处理13-怀柔区/16
怀柔区 区域内,工商银行-金融个数：17,总页数1
处理13-怀柔区/16完成，工商银行-金融 数据集维度：(1276, 11)
开始处理14-平谷区/16
平谷区 区域内,工

In [36]:
results

,address,area,city,detail,lat,lng,name,province,street_id,telephone,uid
0,北京市东城区崇文门外大街90号,东城区,北京市,1,39.901213,116.425594,中国工商银行(崇文门外大街支行),北京市,fc7675244b9b7b874e776ecb,(010)67114823,fc7675244b9b7b874e776ecb
1,北京市东城区张自忠路10号华人一品大厦1层,东城区,北京市,1,39.939357,116.418112,中国工商银行(北京平安支行),北京市,cc788fde61b6c18888fc0665,(010)64078708,cc788fde61b6c18888fc0665
2,东安门大街甲64号附近,东城区,北京市,1,39.921372,116.414488,中国工商银行,北京市,0a633a446fc210f417c8f692,95588,0a633a446fc210f417c8f692
3,北京市东城区建国门内大街乙18号院1号楼1层,东城区,北京市,1,39.913227,116.430288,中国工商银行(北京中电财支行),北京市,b869b18dc8552c0ed42ce246,"(010)65218062,(010)65218057",b869b18dc8552c0ed42ce246
4,北京市东城区柳罐胡同甲2号,东城区,北京市,1,39.911695,116.440193,中国工商银行(北京站支行),北京市,cd9d05ddfb88feb685420ec9,"(010)65238692,(010)65263920,(010)65263921",cd9d05ddfb88feb685420ec9
5,北京市东城区和平里北街14号一层,东城区,北京市,1,39.965124,116.432535,中国工商银行(和平里北街支行),北京市,c8e8f63317a59112ab0dd8cc,(010)84216097,c8e8f63317a59112ab0dd8cc
6,东长安街1号东方广场E3座1层,东城区,北京市,1,39.916141,116.423552,中国工商银行(北京东长安街支行),北京市,24d05508ea8ccbd2128824de,(010)85181650,24d05508ea8ccbd2128824de
7,北京市东城区东直门外新中街甲2号,东城区,北京市,1,39.946405,116.445800,中国工商银行(北京东直门支行),北京市,dc0f0adc6a070921f82213ff,(010)51203011,dc0f0adc6a070921f82213ff
8,工人体育场北路66号瑞士公寓1楼,东城区,北京市,1,39.939256,116.443902,中国工商银行(北京工体北路支行),北京市,71768acfdabe715b9d95de2d,(010)64015062,71768acfdabe715b9d95de2d
9,北京市东城区南竹杆胡同6号北京INN三号楼底商,东城区,北京市,1,39.927568,116.438243,中国工商银行(北京朝内大街支行),北京市,5e2cae62d3de79b5077ce800,(010)65175651,5e2cae62d3de79b5077ce800


In [40]:
url = 'http://api.map.baidu.com/place/v2/search?query={}&tag={}&region={}&page_size={}&page_num={}&output=json&ak={}'.format(
   keyword,
   tag,
   region,
   page_size,
   page_num,
   ak)
url # display URL

'http://api.map.baidu.com/place/v2/search?query=农业银行&tag=金融&region=怀柔区&page_size=20&page_num=0&output=json&ak=10ykqahdsmqaYOkrLgPzNNc6QAD2HOSZ'

In [55]:
while True:
    try:
        results = requests.get(url).json()
        if results['status']==0:
            break
    except:
        print("调用接口失败")

In [62]:
get_venues_num(keyword,tag,region,page_size,page_num,ak)

19

In [46]:
#results = requests.get(url).json()
results

{'status': 0,
 'message': 'ok',
 'total': 21,
 'results': [{'name': '中国农业银行(北京怀柔东环支行)',
   'location': {'lat': 40.33447, 'lng': 116.650679},
   'address': '北京市怀柔区开放路99号',
   'province': '北京市',
   'city': '北京市',
   'area': '怀柔区',
   'street_id': 'b9e5d0b8344d39bc81a12e72',
   'telephone': '(010)69641626',
   'detail': 1,
   'uid': 'b9e5d0b8344d39bc81a12e72'},
  {'name': '中国农业银行(北京城北支行)',
   'location': {'lat': 40.340249, 'lng': 116.63708},
   'address': '北京市怀柔区富乐小区北里25号',
   'province': '北京市',
   'city': '北京市',
   'area': '怀柔区',
   'street_id': 'd1469ee75f133e6384330474',
   'telephone': '(010)69645710',
   'detail': 1,
   'uid': 'd1469ee75f133e6384330474'},
  {'name': '中国农业银行(北京湖光支行)',
   'location': {'lat': 40.332219, 'lng': 116.637282},
   'address': '北京市怀柔区湖光小区青春路甲54号',
   'province': '北京市',
   'city': '北京市',
   'area': '怀柔区',
   'street_id': 'e0f871002a2feacfc16c144d',
   'telephone': '(010)69643717',
   'detail': 1,
   'uid': 'e0f871002a2feacfc16c144d'},
  {'name': '中国农业银行(北京红螺支行)

In [42]:
results['status']

0

In [35]:
venues_num=results['total']
page_num_all=venues_num//20 + 1
print("{} 区域内,{}-{}个数：{},总页数{}".format(region,keyword,tag,venues_num,page_num_all))
#venues=results['results']

怀柔区 区域内,农业银行-金融个数：21,总页数2


In [37]:
results = pd.DataFrame()
for num in range(page_num_all):
    print("完成第{}页".format(num))
    url = 'http://api.map.baidu.com/place/v2/search?query={}&tag={}&region={}&page_size={}&page_num={}&output=json&ak={}'.format(
       keyword,
       tag,
       region,
       page_size,
       num,
       ak)
    result = requests.get(url).json()
    time.sleep(5)
    venues=result['results']
    nearby_venues = json_normalize(venues) # flatten JSON
    #nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
    results=pd.concat([results,nearby_venues],ignore_index=True)
    print(results.shape)
    

完成第0页
(20, 11)
完成第1页
(21, 11)


/home/machinelearning/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  app.launch_new_instance()


In [38]:
results.columns = [col.split(".")[-1] for col in results.columns]

In [39]:
results

,address,area,city,detail,lat,lng,name,province,street_id,telephone,uid
0,北京市怀柔区开放路99号,怀柔区,北京市,1,40.334470,116.650679,中国农业银行(北京怀柔东环支行),北京市,b9e5d0b8344d39bc81a12e72,(010)69641626,b9e5d0b8344d39bc81a12e72
1,北京市怀柔区富乐小区北里25号,怀柔区,北京市,1,40.340249,116.637080,中国农业银行(北京城北支行),北京市,d1469ee75f133e6384330474,(010)69645710,d1469ee75f133e6384330474
2,北京市怀柔区湖光小区青春路甲54号,怀柔区,北京市,1,40.332219,116.637282,中国农业银行(北京湖光支行),北京市,e0f871002a2feacfc16c144d,(010)69643717,e0f871002a2feacfc16c144d
3,富乐大街26号院11,怀柔区,北京市,1,40.347739,116.637551,中国农业银行(北京红螺支行),北京市,92b806d2ad46ec8379b55c20,(010)89688975,92b806d2ad46ec8379b55c20
4,北京市怀柔区庙城镇庙城村510号1幢1至2层,怀柔区,北京市,1,40.299453,116.634530,中国农业银行(北京庙城支行),北京市,c7eca12b84e69936e115f93e,(010)60693257,c7eca12b84e69936e115f93e
5,北京市怀柔区府前东街12号楼7至8号,怀柔区,北京市,1,40.322605,116.647051,中国农业银行(北京迎宾支行),北京市,2bcbc54cdb894e3c2202208b,95599,0ce65c2d6b6f7d592d473372
6,北京市怀柔区金台园甲56号,怀柔区,北京市,1,40.327879,116.645463,中国农业银行(北京兴怀大街支行),北京市,442076466958d516deee8b23,(010)69644532,442076466958d516deee8b23
7,北京市怀柔区,怀柔区,北京市,1,40.415641,116.695159,中国农业银行(北京怀北庄支行),北京市,NaN,NaN,62425a06a095bf8413293555
8,北京市怀柔区雁栖经济开发区雁栖路31号,怀柔区,北京市,1,40.358444,116.690489,中国农业银行(北京雁栖开发区支行),北京市,9e8cea88d259c725c51b75ca,(010)61665856,9e8cea88d259c725c51b75ca
9,北京市怀柔区大星发农发地农副产品批发市场,怀柔区,北京市,1,40.342267,116.652559,中国农业银行ATM,北京市,NaN,NaN,15bd45f5c7ae5b6cf603b807


In [117]:
results[results['area']=='海淀区']

,address,area,city,detail,lat,lng,name,province,street_id,telephone,uid
1,北京市海淀区清河镇龙岗路20号,海淀区,北京市,1,40.036410,116.362553,中国农业银行(北京龙岗路支行),北京市,f448d513e6c91b620d3dda4b,"(010)62991550,(010)62996623",f448d513e6c91b620d3dda4b
2,北京市海淀区花园路甲2号院4号楼一层101,海淀区,北京市,1,39.986153,116.372868,中国农业银行(北京花园路支行),北京市,a2b093525238353000ec0120,(010)82247102,a2b093525238353000ec0120
8,海淀区蓝靛厂中路晨月园甲4号,海淀区,北京市,1,39.964899,116.290373,中国农业银行(北京世纪城支行),北京市,606289fd39b75f4ca304648f,NaN,606289fd39b75f4ca304648f
12,北京市海淀区北蜂窝路5号院2号楼,海淀区,北京市,1,39.910947,116.334905,中国农业银行(北京铁道支行),北京市,befdb292e60ee99c8715413b,NaN,befdb292e60ee99c8715413b
41,北京市海淀区西郊板井曙光花园中路11号北京农科大厦首层B103号,海淀区,北京市,1,39.949037,116.293928,中国农业银行(北京曙光支行),北京市,4642844a215104d9c9fa5f41,NaN,4642844a215104d9c9fa5f41
42,北京市海淀区西海国际中心1层,海淀区,北京市,1,39.969957,116.315115,中国农业银行(北京西海支行),北京市,548aa32bae1e6276c42ba11b,(010)82194280,548aa32bae1e6276c42ba11b
66,北京市海淀区清河小营西路,海淀区,北京市,1,40.042356,116.341820,中国农业银行(北京清河支行),北京市,0752d427705939723a13fd9c,(010)62911826,0752d427705939723a13fd9c
67,北京市海淀区马连洼北路东居物业写字楼一层,海淀区,北京市,1,40.039892,116.293431,中国农业银行(北京竹园支行),北京市,25ec69fc59ab547dc26c146c,(010)82899445,25ec69fc59ab547dc26c146c
68,北京市海淀区上地三街9号嘉华大厦a座1楼,海淀区,北京市,1,40.042110,116.313938,中国农业银行(北京上地支行),北京市,91c13f93bba2cffa21f0176c,(010)62967285,91c13f93bba2cffa21f0176c
91,北京市海淀区复兴路28号中国人民解放军总医院门诊楼一层东侧2区房,海淀区,北京市,1,39.912694,116.283432,中国农业银行(五棵松储蓄所),北京市,a892d50d4830149f6f3e1b8a,NaN,a892d50d4830149f6f3e1b8a


In [72]:
nearby_venues = json_normalize(venues) # flatten JSON

In [73]:
#nearby_venues

In [74]:
# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
print(nearby_venues.shape)

nearby_venues

(20, 11)


,address,area,city,detail,lat,lng,name,province,street_id,telephone,uid
0,北京市朝阳区望京商业街首层A01,朝阳区,北京市,1,40.014711,116.473317,中国农业银行(北京望京支行),北京市,5eca57d48fd9286a9e9d685c,(010)64711745,5eca57d48fd9286a9e9d685c
1,北京市朝阳区杨闸环岛朗廷大厦1层,朝阳区,北京市,1,39.917008,116.605053,中国农业银行(北京八里桥支行),北京市,53cebd504a6afc602a02e162,NaN,53cebd504a6afc602a02e162
2,北京市朝阳区新源南路13号琨莎中心东侧,朝阳区,北京市,1,39.953933,116.459226,中国农业银行(北京新源里支行),北京市,e6ee2fd2d682fc362a11d1b6,(010)64675046,e6ee2fd2d682fc362a11d1b6
3,北京市朝阳区太阳宫南街13号楼1层,朝阳区,北京市,1,39.980271,116.450977,中国农业银行(北京太阳宫支行),北京市,12741f2408eafe8df4b9163b,(010)84150969,12741f2408eafe8df4b9163b
4,花家地街1号,朝阳区,北京市,1,39.991181,116.476041,中国农业银行(北京将台路支行),北京市,cc788fde05982d8a88fc064c,(010)64722757,cc788fde05982d8a88fc064c
5,北京市朝阳区麦子店街78号,朝阳区,北京市,1,39.950969,116.478695,中国农业银行(北京康运支行),北京市,78bd50f75112585b31870a4c,"95599,(010)65042614",78bd50f75112585b31870a4c
6,北京市朝阳区霄云路32号,朝阳区,北京市,1,39.963700,116.470767,中国农业银行(北京三元支行),北京市,3a0a78bdd1a317fa6318094c,(010)64624386,3a0a78bdd1a317fa6318094c
7,北京市朝阳区朝阳北路219号天伦伟业大厦一层及二层,朝阳区,北京市,1,39.929329,116.472553,中国农业银行(北京水碓西里支行),北京市,e3e2cd9d8089bbe3c7f50d4c,(010)85983473,e3e2cd9d8089bbe3c7f50d4c
8,北京市朝阳区红军营南路辅路,朝阳区,北京市,1,40.039250,116.422294,中国农业银行(北京北苑支行),北京市,NaN,NaN,55290d03c0947d03581093aa
9,北京市朝阳区宏昌路望京西园二区221号,朝阳区,北京市,1,40.007981,116.479234,中国农业银行(北京望盛园支行),北京市,4edf9fbff0991394cc06a47a,(010)64789601,4edf9fbff0991394cc06a47a
